## **Personal Expenses Data Preparation**

### **Read data from CSV**  
Export data from the smartphone app I use to collect my expense data. The data comes in a handy CSV format, so I can easily load it into a pandas DataFrame by specifying a delimiter. ~~I also specify other parameters: which columns to load and parsing dates from the 'date' column.~~

In [ ]:
import pandas as pd
import numpy as np

fname = "data/report_2022-10-16_090806.csv"
# load the data
df = pd.read_csv(
    fname,
    sep=";",
    usecols=[
        "account",
        "category",
        "currency",
        "amount",
        "ref_currency_amount",
        "type",
        "payment_type",
        "payment_type_local",
        "note",
        "date",
        "labels"
    ],
    parse_dates=["date"]
)
df.head()


### **Data Cleaning and Preparation**

#### **Check for duplicated and missing data**

In [ ]:
# check non-null count and dtype of each variable
df.info()


Note and labels variables contain missing values. These fields are optional when I create entries in the app and will not impact the accuracy of the analysis. I will fill empty values with "NA".

In [ ]:
# fill NaN values with "NA"
df.fillna('NA', inplace=True)

Check for duplicated rows and remove if any. 


In [ ]:
# get all duplicated values
df[df.duplicated(keep=False)]

There is actually one duplicated entry, so I will remove one.

In [ ]:
# remove duplicated rows
df.drop_duplicates(inplace=True)

#### **Transform, add additional variables**

The variable "Category" actually contains subcategory entries. I will add an additional variable that contains category values.  
I will combine this step with adding the nature of the expense -- [need, want].  
~~In this step, I add the category names. The exported data set doesn't contain this data, so I copied it manually from the application and created a dictionary(***category : subcategory***)
After that, I will map the category value to each row based on the subcategory using pandas **map()**.~~

In [ ]:
# get distinct subcategories
subcategories = df['category'].unique().tolist()

In [ ]:
# create dict with categories as key and subcategory as value and nature [need, want]

d = {
    "Food and Drinks": [
        "Food & Drinks",
        "Bar, cafe",
        "Groceries",
        "Restaurant, fast-food",
        "Fitness Supplements",
        "Coffee",
        "Eating out"
    ],
    "Shopping": [
        "Shopping",
        "Clothes & shoes",
        "Drug-store, chemist",
        "Electronics, accessories",
        "Camera expenses",
        "Free time",
        "Gifts, joy",
        "Health and beauty",
        "Teeth care",
        "Skincare face",
        "Supplements",
        "Medicine",
        "Home, garden",
        "Jewels, accessories",
        "Stationery, tools",
    ],
    "Housing": ["Housing", "Energy, utilities", "Maintenance, repairs", "Rent"],
    "Transportation": [
        "Transportation",
        "Business trips",
        "Long distance",
        "Public transport",
        "Taxi",
    ],
    "Vehicle": [
        "Vehicle",
        "Fuel",
        "Leasing",
        "Parking",
        "Rentals",
        "Vehicle insurance",
        "Vehicle maintenance",
    ],
    "Life and Entertainment": [
        "Life & Entertainment",
        "Active sport, fitness",
        "Alcohol, tobacco",
        "Books, audio, subscriptions",
        "Charity, gifts",
        "Culture, sport events",
        "Education, development",
        "Health care, doctor",
        "Hobbies",
        "Holiday, trips, hotels",
        "Sightseeing, activities",
        "Accommodation",
        "Life events",
        "Lottery, gambling",
        "TV, Streaming",
        "Wellness, beauty",
    ],
    "Communication and PC": [
        "Communication, PC",
        "Internet",
        "Phone, mobile phone",
        "Postal services",
        "Software, apps, games",
        "Phone, cell phone",
    ],
    "Financial Expenses": [
        "Financial expenses",
        "Advisory",
        "Charges, Fees",
        "Fines",
        "Insurances",
        "Loan, interests",
        "Taxes",
    ],
    "Investments": [
        "Investments",
        "Financial investments",
        "Collections",
        "Realty",
        "Savings",
        "Vehicles, chattels",
    ],
    "Income": ["Income", "Gifts", "Refunds (tax, purchase)", "Sale", "Wage, invoices", "Lending, renting", "Rentals"],
    "Other": ["Missing", "Other"],
}

d_nat = {
    "need": [
        "Food & Drinks",
        "Groceries",
        "Restaurant, fast-food",
        "Clothes & shoes",
        "Drug-store, chemist",
        "Teeth care",
        "Supplements",
        "Medicine",
        "Home, garden",
        "Housing",
        "Energy, utilities",
        "Maintenance, repairs",
        "Rent",
        "Transportation",
        "Long distance",
        "Public transport",
        "Taxi",
        "Active sport, fitness",
        "Communication, PC",
        "Internet",
        "Phone, mobile phone",
        "Postal services",
        "Phone, cell phone",
        "Charges, Fees",
        "Fines",
        "Insurances",
        "Loan, interests",
        "Taxes",
        "Other",
        "Missing",
        "Housing",
        'Financial expenses',
    ],

    'want': [
        "Bar, cafe",
        "Fitness Supplements",
        "Coffee",
        "Eating out"
        "Shopping",
        "Electronics, accessories",
        "Camera expenses",
        "Free time",
        "Gifts, joy",
        "Health and beauty",
        "Skincare face",
        "Skincare body"
        "Jewels, accessories",
        "Stationery, tools",
        "Business trips",
        "Vehicle",
        "Fuel",
        "Leasing",
        "Parking",
        "Rentals",
        "Vehicle insurance",
        "Vehicle maintenance",
        "Life & Entertainment",
        "Alcohol, tobacco",
        "Books, audio, subscriptions",
        "Charity, gifts",
        "Culture, sport events",
        "Education, development",
        "Health care, doctor",
        "Hobbies",
        "Holiday, trips, hotels",
        "Sightseeing, activities",
        "Accommodation",
        "Life events",
        "Lottery, gambling",
        "TV, Streaming",
        "Wellness, beauty",
        "Software, apps, games",
        "Advisory",
        'Shopping'
    ]
}


In [ ]:
# define a function to flatten dict
def flatten_dict(d):
    """This function flattens dictionaries"""
    nd = {}
    for k, v in d.items():
        # Check if it's a list, if so then iterate through
        if hasattr(v, "__iter__") and not isinstance(v, str):
            for item in v:
                nd[item] = k
        else:
            nd[v] = k
    return nd


In [ ]:
# flatten the category and nature dictionaries
flatten_d = flatten_dict(d)
flatten_d_nat = flatten_dict(d_nat)


In [ ]:
# rename the category to subcategory
df = df.rename(columns={'category': 'subcategory'})


In [ ]:
# map the values from dictionaries to corresponding values in data frame
df["category"] = df["subcategory"].map(flatten_d)
df['nature'] = df['subcategory'].map(flatten_d_nat)
df.head()


In [ ]:
# convert the amount variables to absolute values
df[["amount", "ref_currency_amount"]] = df[[
    "amount", "ref_currency_amount"]].abs()
df.head()


Split the date column to date only and time.

In [ ]:
df['time'] = df['date'].dt.time
df.head()


###### **Split the *Labels* column to 3 columns as it contains multiple values**.

In [ ]:
df[["l1", "l2", "l3", "l4"]
         ] = df["labels"].str.rsplit("|", expand=True)
df[["l1", "l3", "l3", "l4"]]


The values are mixed across these 4 label columns. I convert these Series to lists to bring the values in correct place. 

In [ ]:
# save the the splitted columns to lists to iterate and change the values
list_1 = df["l1"].to_list()
list_2 = df["l2"].to_list()
list_3 = df["l3"].to_list()
list_4 = df["l4"].to_list()


In [ ]:
# get unique values (these are the place names)
places = list(df["l3"].unique())

In [ ]:
# create a list with invalid names or NaN values
del_place = [1, 2, 6, 18]
# remove and using numpy and convert back to list
places_1 = np.delete(places, del_place).tolist()


In [ ]:
# iterate through list_3 -- there are the majority of correct values.
# Iterate through it and if the value is not in the list with correct places
# look in other columns and append to a new list
nvalid = ("BIG TRIP", "Thailand")
place = []
for x in list_3:
    if x in places_1:
        place.append(x)
    elif x in nvalid and list_2[list_1.index(x)] in nvalid:
        place.append(list_1[list_3.index(x)])
    elif x in nvalid and list_1[list_3.index(x)] in nvalid:
        place.append(list_2[list_1.index(x)])
    elif x == "Accommodation":
        x = list_4[list_3.index(x)]
        place.append(x)
    else:
        place.append(x)


In [ ]:
# append the new list to the data frame
df["place"] = place
df.head()


In [48]:
# exclude/filter out deposit entries (deposits for hotel rooms etc.)
df = df[~df.note.str.contains("Deposit")]

In [49]:
# fill NaN values in nature column to NA (for income entries)
df['nature'] = df['nature'].fillna(value='NA')

In [51]:
# create new column
df[['country', 'lat', 'lng']] = 'NA'

In [ ]:
# split the data before and during travel
start_date = pd.datetime(2021, 10, 2)
end_date = pd.datetime(2022, 10, 24)

home_df = df.loc[df["date"] < start_date]
travel_df = df.loc[(df["date"] >= start_date) & (df["date"] <= end_date)]

non_travel_exp = [
    "Camera Expenses",
    "Electronics, accessories",
    "Books, audio, subscriptions",
    "Education, development",
]

trip_label = "BIG TRIP"


In [ ]:
# assign conditional column 
home_df['travel_expense'] = np.where(home_df['labels'].str.contains(trip_label), True, False)
travel_df['travel_expense'] = np.where(travel_df['subcategory'].isin(non_travel_exp), False, True)

In [ ]:
# inspect the travel df for missing values in place column
travel_df[travel_df['travel_expense'] == True]

In [ ]:
# combine the datasets again
df = pd.concat([home_df, travel_df])
df.head()

In [ ]:
# fill na in place column with ffill method (forward fill)
dftravel["place"].fillna(method="ffill", inplace=True)
dftravel.info()


In [ ]:
# change values that were not correctly filled in previous step
dftravel.loc[dftravel["place"] == "Accommodation", ["place"]] = "Phuket"
dftravel.loc[dftravel["place"] == "Road trip", ["place"]] = "Sangkhlaburi"
dftravel.loc[dftravel["place"] == "BIG TRIP", ["place"]] = "Bangkok"


In [ ]:
# create a new column 'country'
dftravel["country"] = "Thailand"


In [ ]:
# finally drop not needed columns
dftravel.drop(["labels", "l1", "l2", "l3", "l4"], axis=1, inplace=True)


In [ ]:
# check summary for each column to spot possible issues
dftravel.info()


#### **Get latitude and longitude for the places**

In [ ]:
import urllib.request
import urllib.parse
import urllib.error
import json
import ssl

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'
# https://developers.google.com/maps/documentation/geocoding/intro

if api_key is False:
    api_key = 42
    serviceurl = 'http://py4e-data.dr-chuck.net/json?'
else:
    serviceurl = 'https://maps.googleapis.com/maps/api/geocode/json?'

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

keys = dftravel["place"].unique()
geodata = list()

for place in keys:
    parms = dict()
    parms['address'] = place

    if api_key is not False:
        parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters')

    try:
        js = json.loads(data)
    except:
        js = None

    if not js or 'status' not in js or js['status'] != 'OK':
        print('==== Failure To Retrieve ====')
        print(data)
        continue

        print(json.dumps(js, indent=4))

    lat = js['results'][0]['geometry']['location']['lat']
    lng = js['results'][0]['geometry']['location']['lng']
    geodata.append([lat, lng])
    print('lat', lat, 'lng', lng)
    location = js['results'][0]['formatted_address']
    print(location)


In [ ]:
# use the zip function to make a dict from two lists
geo_dict = dict(zip(keys, geodata))
geo_dict


In [ ]:
# and finally map the dict values to the dataframe
dftravel["gdata"] = dftravel["place"].map(geo_dict)
dftravel.head()


In [ ]:
# latitude and longitude are stored in one column, I split the column to two columns
dftravel[["lat", "lng"]] = pd.DataFrame(
    dftravel.gdata.to_list(), index=dftravel.index)
dftravel.drop("gdata", axis=1, inplace=True)
print(dftravel.dtypes)


In [ ]:
# adjust the column order
col_names = dftravel.columns.values.tolist()
col_order = ['date',
             'year',
             'month',
             'day',
             'weekday',
             'time',
             'category',
             'subcategory',
             'nature',
             'amount',
             'account',
             'payment_type',
             'lat',
             'lng',
             'place',
             'country']

dftravel = dftravel.reindex(columns=col_order)
dftravel.head()


#### **Write the data to CSV**

In [ ]:
dftravel.to_csv("data/2022-09-27_travel_expenses.csv", index=False)


#### **Write the data to a SQLite database file.**

In [ ]:
# write data to a SQLite database file
import sqlite3 as sq

sql_data = "data/EXPENSES.db"
conn = sq.connect(sql_data)
cur = conn.cursor()
cur.execute("""DROP TABLE IF EXISTS travel_expenses""")
dftravel.to_sql("dftravel", conn, if_exists="replace", index=False)
conn.commit()
conn.close()
